In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import pickle
import nltk

from torchtext.vocab import GloVe

# In this Notebook, I will preprocess the texts with GloVe and store the resulting vectors for usage in other Notebooks.
(with and without stopword removal)

## Load train and test data

In [ ]:
train = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")

# Load the GloVe pretrained embedding

## Options:

* Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 50d, 100d, 200d, & 300d vectors, ca. 822 MB download):
  * EMBEDDING_NAME = "6B"
  * EMBEDDING_DIM = [50, 100, 200, 300]
* Common Crawl (42B tokens, 1.9M vocab, uncased, 300d vectors, ca. 1.75 GB download): 
  * EMBEDDING_NAME = "42B"
  * EMBEDDING_DIM = 300
* Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, ca. 2.03 GB download): 
  * EMBEDDING_NAME = "840B"
  * EMBEDDING_DIM = 300
* Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, ca. 1.42 GB download):
  * EMBEDDING_NAME = "twitter.27B"
  * EMBEDDING_DIM = [25, 50, 100, 200]

In [ ]:
EMBEDDING_DIM = 300
EMBEDDING_NAME = "6B"

In [ ]:
glove = GloVe(name=EMBEDDING_NAME, dim=EMBEDDING_DIM)

In [ ]:
remove_words = nltk.corpus.stopwords.words('english') # Load standard english stopwords
remove_words.extend([".", ",", "!", "?", "'", ":", ";", '"', "-", "''", '``']) # The nltk tokenizer will also use these as tokens, but I want to remove them


def get_vocab(text, stopword_removal=False):
    words = nltk.tokenize.word_tokenize(text.lower())
    words_clean = [word for word in words]
    if stopword_removal:
        words_clean = [word for word in words_clean if word not in remove_words]
    return words_clean

In [ ]:
# The magic happens here, extract GloVe representations for each word

def get_word_vectors(text, stopword_removal=False, text_len=270):
    words = get_vocab(text, stopword_removal=stopword_removal)
    result = np.zeros((text_len, EMBEDDING_DIM))
    for i, word in enumerate(words):
        result[i,:] = glove[word]
    return result

# Look for the maximum sentence length

In [ ]:
print("Stentence length\n\t* with stopword removal: {}\n\t* without stopword removal: {}".format(train.excerpt.apply(lambda x: len(get_vocab(x, stopword_removal=True))).max(), train.excerpt.apply(lambda x: len(get_vocab(x, stopword_removal=False))).max()))

# First without stopword removal

In [ ]:
def extract_and_save_representation(data, stopword_removal, text_len, filename):
    # extract
    get_word_vectors_lambda = lambda x: get_word_vectors(x, stopword_removal=stopword_removal, text_len=text_len)
    emb_text = data.excerpt.apply(get_word_vectors_lambda)
    emb_text = np.stack(emb_text)
    
    # save
    if "target" in data.columns: # Test data has no target column
        pickle.dump({"id": data.id.values, "target": data.target.values, "wordvectors": emb_text}, open(filename, "wb"))
    else:
        pickle.dump({"id": data.id.values, "wordvectors": emb_text}, open(filename, "wb"))

In [ ]:
# without stopword removal
extract_and_save_representation(train, False, 270, "train_with_stopwords_glove_{}_{}.pkl".format(EMBEDDING_NAME, EMBEDDING_DIM))
extract_and_save_representation(test, False, 270, "test_with_stopwords_glove_{}_{}.pkl".format(EMBEDDING_NAME, EMBEDDING_DIM))

# with stopword removal
extract_and_save_representation(train, True, 143, "train_without_stopwords_glove_{}_{}.pkl".format(EMBEDDING_NAME, EMBEDDING_DIM))
extract_and_save_representation(test, True, 143, "test_without_stopwords_glove_{}_{}.pkl".format(EMBEDDING_NAME, EMBEDDING_DIM))

# would be more clean as loop over configurations

## The output now contains 4 files: Train and test data with and without stopwords